# glTF (Graphics Language Transmission Format)

Material Specification is described in https://registry.khronos.org/glTF/specs/2.0/glTF-2.0.html#appendix-b-brdf-implementation



In [ ]:
from glTF import *

brdf = glTF_brdf()
base_color, alpha, metallic = brdf.material_params
ior = sp.Symbol(r'\mathrm{ior}')
# ior = 1.5 # default without KHR_materials_ior

# Material Structure

    material = mix(dielectric_brdf, metal_brdf, metallic)
         = (1.0 - metallic) * dielectric_brdf + metallic * metal_brdf


## Metals

metal_brdf =
  conductor_fresnel(
    f0 = baseColor,
    bsdf = specular_brdf(
      α = roughness ^ 2))

## Dielectrics

    dielectric_brdf =
        fresnel_mix(
            ior = 1.5,
            base = diffuse_brdf(
            color = baseColor),
            layer = specular_brdf(
            α = roughness ^ 2))

# Diffuse Component

In [ ]:
diffuse = diffuse_component(base_color)
display(sp.Eq(sp.Symbol("LambertianBRDF"), diffuse))

# Specular Component

In [ ]:
H_sym, V_sym, L_sym, N_sym = sp.symbols("H, V, L, N")
display(sp.Eq(H_sym, V_sym + L_sym))

D_sym, VX_sym = sp.symbols("D V_ggx")

display(sp.Eq(D_sym, specular_D_GGX(N_sym, H_sym, alpha)))

display(sp.Eq(VX_sym, specular_V_GGX(V_sym, N_sym, L_sym, alpha)))

display(sp.Eq(sp.Symbol("MicrofacetBRDF"), specular_component(VX_sym, D_sym)))

microfacet_brdf = specular_component(specular_V_GGX(V_sym, N_sym, L_sym, alpha), specular_D_GGX(N_sym, H_sym, alpha)).simplify()
display(sp.Eq(sp.Symbol("MicrofacetBRDF"), microfacet_brdf))


# Dielectric Component

In [ ]:
dielectric_brdf = fresnel_mix(V, H, ior, diffuse, microfacet_brdf)
display(sp.Eq(sp.Symbol("dielectric"), dielectric_brdf))

# Metal Component

In [ ]:
metal_brdf = conductor_fresnel(
    V, H, base_color, microfacet_brdf)
display(metal_brdf)

# full glTF 

In [ ]:
gltf_brdf = mix(dielectric_brdf, metal_brdf, metallic)
display(gltf_brdf)

guarded_gltf_brdf = gltf(V, N, L, base_color, metallic, alpha)
display(guarded_gltf_brdf)


In [ ]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"

from bsdf import *
from glTF import *

theta_v = 15 / 90.0 * np.pi / 2
rho_val = np.array([1.        , 1.        , 0.99906583])
roughness_val = np.sqrt(0.1133)
metallic_val = 1.0

N_val = np.array([0, 0, 1], dtype=np.float32)
V_val = np.array([np.sin(theta_v), 0, np.cos(theta_v)], dtype=np.float32)
brdf = glTF_brdf()
brdf_np = brdf.get_np()

print(
    "Integral:",
    integrate_spherical_function(lambda l: brdf_np(
        V_val, N_val, l, rho_val, roughness_val**2, metallic_val) * np.abs(dot(l, N_val)), 100000),
)
plot_brdf(
    "glTF",
    lambda v, n, l: brdf_np(
        v, n, l, rho_val, roughness_val**2, metallic_val),
    V_val,
    normalize=False,
)